In [2]:
pip install dash

   ---------------------------------------- 0.0/7.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/7.5 MB 653.6 kB/s eta 0:00:12
    --------------------------------------- 0.1/7.5 MB 1.1 MB/s eta 0:00:07
    --------------------------------------- 0.2/7.5 MB 1.2 MB/s eta 0:00:06
   - -------------------------------------- 0.3/7.5 MB 1.4 MB/s eta 0:00:06
   - -------------------------------------- 0.3/7.5 MB 1.5 MB/s eta 0:00:05
   -- ------------------------------------- 0.5/7.5 MB 1.6 MB/s eta 0:00:05
   --- ------------------------------------ 0.6/7.5 MB 1.7 MB/s eta 0:00:05
   --- ------------------------------------ 0.7/7.5 MB 1.8 MB/s eta 0:00:04
   ---- ----------------------------------- 0.8/7.5 MB 1.8 MB/s eta 0:00:04
   ---- ----------------------------------- 0.8/7.5 MB 1.8 MB/s eta 0:00:04
   ---- ----------------------------------- 0.9/7.5 MB 1.6 MB/s eta 0:00:05
   ----- ---------------------------------- 1.0/7.5 MB 1.7 MB/s eta 0:00:04
   ----- --------

In [4]:
import wget

In [6]:
import requests

url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"
response = requests.get(url)

with open("spacex_launch_dash.csv", "wb") as file:
    file.write(response.content)


In [7]:
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown(id='site-dropdown',options=[{'label': 'All Sites', 'value': 'ALL'},{'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                            {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},{'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},{'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'}],
                                            value='ALL', placeholder="Select a launch Site here", searchable=True),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider',min=0, max=10000, step=1000,
                                        marks={0: '0',
                                            2500: '2500',
                                            5000: '5000',
                                            7500: '7500',
                                            10000: '10000'},
                                        value=[min_payload, max_payload]),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site):
    filtered_df = spacex_df[spacex_df['Launch Site']==entered_site]
    if entered_site == 'ALL':
        fig = px.pie(spacex_df, values='class', 
        names='Launch Site', 
        title='Total Success Launches By Site')
        return fig
    else:
        filtered_df = filtered_df['class'].value_counts().rename_axis('class').to_frame('counts').reset_index()
        print(filtered_df)
        fig = px.pie(filtered_df, values='counts', names='class', 
        title=f'Total Success Launches for site {entered_site}')
        return fig

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback( Output(component_id='success-payload-scatter-chart', component_property='figure'),
            [Input(component_id='site-dropdown', component_property='value'), Input(component_id="payload-slider", component_property="value")])
def update_chart(entered_site, payload):
    min_payload = payload[0]
    max_payload = payload[1]
    load_filtered = spacex_df[spacex_df['Payload Mass (kg)'].between(min_payload,max_payload)]
    if entered_site == 'ALL':
        fig = px.scatter(load_filtered, x='Payload Mass (kg)', y='class', color='Booster Version Category',
        title=f'Correlation between Payload and Success for {entered_site}')
        return fig
    else:
        filtered_df = load_filtered[load_filtered['Launch Site']==entered_site]
        fig = px.scatter(filtered_df, x='Payload Mass (kg)', y='class', color='Booster Version Category', 
        title=f'Correlation between Payload and Success for {entered_site}')
        return fig

# Run the app
if __name__ == '__main__':
    app.run_server()

C:\Anaconda\Lib\site-packages\plotly\express\_core.py:1979: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.

C:\Anaconda\Lib\site-packages\plotly\express\_core.py:1979: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.

C:\Anaconda\Lib\site-packages\plotly\express\_core.py:1979: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



   class  counts
0      0      19
1      1       7
   class  counts
0      1      10
1      0       3
